# 角度変化する孤立水分子の計算

## 2022/2/15

## 角度を変化させ，対応するQEのinputファイルを自動生成する．

MDの.posファイルからH2Oの座標を取得して，それを出力ファイルに書き出す．

In [1]:
import numpy as np

In [3]:
# ファイルを読み込み．
zahixyou=np.loadtxt("h2o_mol2.pos")
print(zahixyou)

print(zahixyou[44])

[[ 9.90000000e-03  9.90000000e-03  0.00000000e+00]
 [ 1.82896845e+00 -2.24652669e-01  2.92522173e-03]
 [-2.23534394e-01  1.83033378e+00  6.98697185e-04]
 [ 9.90000000e-03  9.90000000e-03  0.00000000e+00]
 [ 1.82605464e+00 -2.24278426e-01  6.18214549e-03]
 [-2.22903058e-01  1.82714531e+00  1.12768217e-03]
 [ 9.90000000e-03  9.90000000e-03  0.00000000e+00]
 [ 1.83004965e+00 -2.23068650e-01  4.03081698e-03]
 [-2.22467650e-01  1.82646432e+00  1.54234565e-03]
 [ 9.90000000e-03  9.90000000e-03  0.00000000e+00]
 [ 1.82989765e+00 -2.22664084e-01  2.47327874e-03]
 [-2.21480217e-01  1.82556166e+00  1.55015138e-03]
 [ 9.90000000e-03  9.90000000e-03  0.00000000e+00]
 [ 1.82512200e+00 -2.22402873e-01  6.42737059e-03]
 [-2.20884316e-01  1.82414644e+00  1.01918162e-03]
 [ 9.90000000e-03  9.90000000e-03  0.00000000e+00]
 [ 1.82891419e+00 -2.22361645e-01  1.01984146e-02]
 [-2.21249473e-01  1.82797176e+00  2.29160718e-03]
 [ 9.90000000e-03  9.90000000e-03  0.00000000e+00]
 [ 1.83280292e+00 -2.22206815e-

In [8]:
#O，H1，H2はいずれも3次元の座標
def make_file(O,H1,H2):
    lines= [
    "&control", 
    "      calculation = 'scf'", 
    "      prefix = 'h2o_bec'", 
    "      pseudo_dir = '/home/amano/src/qe-6.4.1/pseudo/'", 
    "      outdir           = './work/'", 
    " ",    
    "/", 
    "&system", 
    "      ibrav=1, celldm(1) =10,", 
    "      nat=3, ntyp=2,", 
    "      ecutwfc=70.0", 
    "      nosym =.TRUE.", 
    "/", 
    "&electrons", 
    "      conv_thr=1e-8", 
    "/", 
    "&ions", 
    "/", 
    "ATOMIC_SPECIES", 
    "H   1.00784 H.pbe-rrkjus_psl.1.0.0.UPF", 
    "O  15.999 O.pbe-n-rrkjus_psl.1.0.0.UPF", 
    " ",
    " " ,  
    "ATOMIC_POSITIONS (angstrom)", 
    " O {0} {1} {2}".format(O[0], O[1], O[2]), 
    " H {0} {1} {2}".format(H1[0], H1[1], H1[2]), 
    " H {0} {1} {2}".format(H2[0], H2[1], H2[2]), 
    " " , 
    "K_POINTS (automatic)", 
    " 1 1 1 0 0 0", 
    ]
    #出力ファイル名
    output="h2o_bec.pw.in"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))


In [9]:
#phファイルの方
def make_phfile():
    lines=[
    "h2o phonon",
    "&inputph",
    "     tr2_ph = 1.0d-14," ,
    "     prefix = 'h2o_bec'" ,
    "     outdir = './work/'"  ,
    "     trans  = .true.", 
    "     zue    = .true.", 
    "     epsil  = .true.", 
    "   asr      = .true.",    
    "/",
    "0 0 0",
    ]
    #出力ファイル名
    output="h2o_bec.ph.in"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))


In [10]:
#copy.shの作成
def make_copyfile():
    lines=[
    "#!/bin/sh",
    "number=0",
    "while test $number -le 14",
    "do",
    "      cp sample4.slurm ./water_$number/",
    "      number=`expr $number + 1`",
    "done",
    ]
    #出力ファイル名
    output="copy.sh"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))


In [11]:
# wan.shの作成
# ここで結晶構造も必要なので場合によって書き換えること！！
def make_wanfile(O,H1,H2):
        lines=[
            "# num_wann sould be equal to nbnd",
            "# default nbnd in insulater is electron num/2",
            "num_wann        = 4",
            "num_iter        = 100",
            " ",
            "translate_home_cell=true",
            " ",
            "guiding_centres = true",
            " ",
            "# level of verbosity of the output from 0 to 3(high)",
            "iprint          = 2",
            " ",
            "wannier_plot=.true.",
            "# wannier function output format",
            "wannier_plot_format = cube",
            "wannier_plot_mode = molecule",
            " ",
            " ",
            "begin atoms_cart",
            "ang",
            " O {0} {1} {2}".format(O[0], O[1], O[2]), 
            " H {0} {1} {2}".format(H1[0], H1[1], H1[2]), 
            " H {0} {1} {2}".format(H2[0], H2[1], H2[2]), 
            "end atoms_cart",
            " ",
            "begin projections",
            "random",
            "end projections",
            " ",
            "# ang or bohr",
            "begin unit_cell_cart",
            "bohr",
            "12.00000000000   0.00000000000000 0.00000000000000",
            "0.00000000000000 12.0000000000000 0.00000000000000",
            "0.00000000000000 0.00000000000000 12.0000000000000",
            "end unit_cell_cart",
            " ",
            "mp_grid    : 1 1 1",
            "gamma_only : true",
            " ",
            "begin kpoints",
            "0.0 0.0 0.0",
            "end kpoints",
        ]
        #出力ファイル名
        output="h2o_bec.win"
        #ファイルの保存
        with open(output, mode='w') as f:
            f.write('\n'.join(lines))


In [12]:
# pw2wanファイルの作成
def make_pw2wan():
    lines=[
    "&inputpp",
    "outdir         = './work/'",
   "prefix         = 'h2o_bec'",
   "seedname       = 'h2o_bec'",
   "spin_component = 'none'",
   "write_mmn      = .true.",
   "write_amn      = .true.",
   "write_unk      = .true.",
   "/",
    ]
    #出力ファイル名
    output="h2o_bec.pw2wan"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))



In [13]:
# スクリプトの実行用ファイル
#slurm.shの作成
def make_jobfile(SIZE):
    lines=[
    "#!/bin/sh",
    "number=0",
    " ",
    "# 現在のdirを取得",
    "CURRENT_DIR=$(pwd)",
    " ",
    "while test $number -le {0}".format(SIZE),
    "do",
    " ",
    "    #working dirへ移動",
    "    cd ${CURRENT_DIR}/water_$number/",
    " ",
    "    #jobファイルの実行",  # modify here
    "    sbatch sample4.slurm",
    " ",
    "    number=`expr $number + 1`",
    "done",
    ]
    #出力ファイル名
    output="job.sh"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))



In [14]:
#filenameファイルのディレクトリを移動する．
# shutil.copyfileはscrのファイルをcopyにコピーする．
def copyfile1(num,filename):
    import shutil
    copy = './water_{}'.format(str(num)) #移動先のdir
    shutil.move(filename,copy)

#water_10などのdirを作成
def make_dir(new_dir_path):
    import os
    import shutil
    if os.path.exists(new_dir_path)==True:
        shutil.rmtree(new_dir_path) # すでに存在すれば削除
    os.mkdir(new_dir_path)

In [15]:
# 実際にファイル群を作成する関数
def make_files(zahixyou,SIZE,NUM_MOLECULE):
    for step in np.arange(SIZE):
        print("ステップ：：", step)

        #water_stepのdir作成
        make_dir("water_{0}".format(step))
    
        #必要ファイル作成
        make_scffile(zahixyou, NUM_MOLECULE, step)
        make_phfile()
        make_wanfile(zahixyou, NUM_MOLECULE, step)
        make_pw2wanfile()
        make_scriptfile()
    
        #ファイル移動
        copyfile1(step,"h2o_bec.pw.in")
        copyfile1(step,"h2o_bec.pw2wan")
        copyfile1(step,"h2o_bec.ph.in")
        copyfile1(step,"h2o_bec.win")
        copyfile1(step,"script.sh")

    # 最後にjobファイルの作成
    make_jobfile(SIZE)

In [16]:
# 水の角度変化に対応した座標の計算
#
# 長さはdefaultで0.9693 ang.（relaxしたもの）

def angle_cordinate(angle, long):
    #
    print("angle = ", angle, "   [度]")
    print("long = ", long ,    "   Angstrom")
    # H2の座標を計算
    cord_O  =np.array([0.00, 0.00, 0.00])
    cord_H1=np.array([long, 0.00, 0.00])
    cord_H2=np.array([long*np.cos(angle*np.pi/180), long*np.sin(angle*np.pi/180), 0.00])
    #
    return cord_O, cord_H1, cord_H2


In [26]:
# 実際にファイル群を作成
#

## 角度のリストを作成
list=np.arange(102,130)
print(list)
#
## longの指定
long=0.9693 
#
for angle in list:
    print("ステップ：：", angle)
    
    # 3つの座標の取得
    O, H1, H2=angle_cordinate(angle, long)
    
    #water_angleのdir作成
    make_dir("water_"+str(int(angle)))
    print("water_"+str(int(angle)))
    
    #pwファイル作成
    make_file(O,H1,H2)
    
    #pwファイル移動
    copyfile1(str(int(angle)),"h2o_bec.pw.in")
    
    #phファイル作成
    make_phfile()
    
    #phファイル移動
    copyfile1(str(int(angle)),"h2o_bec.ph.in")
    
    #wanファイル作成
    #make_wanfile(O,H1,H2)
    #wanファイル移動
    #copyfile3(step)
    #pw2wanファイル作成
    #make_pw2wan()
    
    
# 最後にcopy,slurm,jobファイルの作成
make_copyfile()
#make_slurmfile()
make_copyfile()

[102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129]
ステップ：： 102
angle =  102    [度]
long =  0.9693    Angstrom
water_102
ステップ：： 103
angle =  103    [度]
long =  0.9693    Angstrom
water_103
ステップ：： 104
angle =  104    [度]
long =  0.9693    Angstrom
water_104
ステップ：： 105
angle =  105    [度]
long =  0.9693    Angstrom
water_105
ステップ：： 106
angle =  106    [度]
long =  0.9693    Angstrom
water_106
ステップ：： 107
angle =  107    [度]
long =  0.9693    Angstrom
water_107
ステップ：： 108
angle =  108    [度]
long =  0.9693    Angstrom
water_108
ステップ：： 109
angle =  109    [度]
long =  0.9693    Angstrom
water_109
ステップ：： 110
angle =  110    [度]
long =  0.9693    Angstrom
water_110
ステップ：： 111
angle =  111    [度]
long =  0.9693    Angstrom
water_111
ステップ：： 112
angle =  112    [度]
long =  0.9693    Angstrom
water_112
ステップ：： 113
angle =  113    [度]
long =  0.9693    Angstrom
water_113
ステップ：： 114
angle =  114    [度]
long =  0.9693    Angstrom
water_114
ス